<a href="https://colab.research.google.com/github/andrerodrig/cognitive-computing-projects/blob/main/computer_vision/notebooks/Visao_Computacional_Atlantico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from skimage.filters import roberts, sobel, scharr, prewitt

# Padrão ouro da maior e menor largura dos exemplos definidos no padrão ouro de segmentação.

In [ ]:
area = 0
for linha in range(60):
  for coluna in range(60):
    if image_binary[linha,coluna]!= 0: area += 1

In [ ]:
larguras = []
for linha in range(60):
  largura = 0
  for coluna in range(60):
    if image_binary[linha,coluna]!= 0: largura += 1
  larguras.append(largura)

print("A folha possui as larguras:")
print(larguras)

In [ ]:
comprimentos = []
for coluna in range(60):
  comprimento = 0
  for linha in range(60):
    if image_binary[linha,coluna]!= 0: comprimento += 1
  comprimentos.append(comprimento)

print("A folha possui as comprimentos:")
print(comprimentos)

# Armazenar as informações dos itens 3,4,5 em um .csv

In [ ]:
dados_folha = {
    "area":area,
    "largura":larguras[np.argmax(larguras)],
    "comprimento":comprimentos[np.argmax(comprimentos)]
}

In [ ]:
df = pd.DataFrame([dados_folha])

In [ ]:
df.to_csv(path)

# Implementar 3 técnicas de segmentação e aplicar nos 20 exemplos com padrão ouro definido.

In [ ]:
def plot_and_save(img,maskwater,name_img):
    fig,ax = plt.subplots(1,2, figsize=(10,15))
    fig.suptitle('Imagem original x mask padrão ouro x mask watershed')
    ax[0].imshow(img)
    ax[0].set_title('Imagem original')
    ax[1].imshow(maskwater)
    ax[1].set_title('mask watershed')

    plt.savefig(path_mascara+name_img+'.jpg', bbox_inches='tight',pad_inches = 0)
    plt.imsave(path_mascara+name_img+'filtro'+'.jpg', maskwater)

WaterShed

In [ ]:
def im_write(img2,name_img):
    img  = cv.imread(img2)
    img2 = cv.cvtColor(img,cv.COLOR_BGR2RGB)
    gray = cv.cvtColor(img2,cv.COLOR_RGB2GRAY) # escala de cinza
    ret, thresh = cv.threshold(gray,0,255,cv.THRESH_BINARY_INV+cv.THRESH_OTSU) #pegando threshold
    kernel = np.ones((3,3),np.uint8) # criando o kernel 3 x 3
    opening = cv.morphologyEx(thresh,cv.MORPH_OPEN,kernel, iterations = 2) # aplicando tecnicas de erosão e posteriomente dilatação
    dist_transform = cv.distanceTransform(opening,cv.DIST_L2,5) #
    sure_bg = cv.dilate(opening,kernel,iterations=3)
    ret, sure_fg = cv.threshold(dist_transform,0.7*dist_transform.max(),255,0)
    sure_fg = np.uint8(sure_fg)
    unknown = cv.subtract(sure_bg,sure_fg)
    ret, markers = cv.connectedComponents(sure_fg)
    markers = markers+1
    markers[unknown==255] = 0
    markers = cv.watershed(img,markers)
    img[markers == -1] = [255,0,0]
    #plt.imshow(markers)
    # regex
    #cv.imwrite(path_mascara+name_img+'.jpg',markers)
    plot_and_save(img2,markers,name_img)

Sobel

In [ ]:
def sobel_(img_gray)
  img_sobel = sobel(img_gray)

  return img_sobel



Otsu

In [ ]:
def im_write(img2,name_img):
    img  = cv.imread(img2)
    img2 = cv.cvtColor(img,cv.COLOR_BGR2RGB)
    img2 = cv.resize(img2,(1280,960),interpolation = cv.INTER_AREA)
    gray = cv.cvtColor(img2,cv.COLOR_RGB2GRAY) # escala de cinza
    tipo = cv.THRESH_BINARY_INV + cv.THRESH_OTSU
    limiar, image_binary = cv.threshold(gray , 0, 255, tipo)
    plot_and_save(img2,image_binary,name_img)


# Calcular métricas 𝐼𝑂𝑈 (𝑖𝑛𝑡𝑒𝑟𝑠𝑒𝑐𝑡𝑖𝑜𝑛 𝑜𝑣𝑒𝑟 u𝑛𝑖𝑜𝑛) para avaliação da qualidade da segmentação.

In [ ]:
def IoU_image_seg(img_gray, ground_truth, img_seg):
  """
            Funcao responsavel pela métrica IoU
            Arguments:
              img_gray: np.array -- Uma matriz NumPy que representa a imagem na escala de cinza
                com a forma (num_rows, num_cols, num_channels)
              ground_truth: np.array -- Uma matriz NumPy que representa a imagem na escala de cinza
                com a forma (num_rows, num_cols, num_channels)
              img_seg:  np.array -- Uma matriz NumPy que representa a imagem binarizada
            Return:
              
  """
  result1 = cv2.bitwise_and(img_gray, ground_truth)
  result1 = cv2.cvtColor(result1, cv2.COLOR_BGR2RGB)

  result2 = cv2.bitwise_and(img_gray, img_seg)
  result2 = cv2.cvtColor(result2, cv2.COLOR_BGR2RGB)

  intersection = np.logical_and(result1, result2)
  union = np.logical_or(result1, result2)
  iou_score = np.sum(intersection) / np.sum(union)
  print('IoU is %s' % iou_score)